In [1]:
!pip install yfinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15191 sha256=26db5be5a0b8772d5a6e7569cd4cb45269b586c6ddda2b6d4344f4cb806b0a89
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [2]:
import numpy as np
import time as tm
import datetime 
import tensorflow as tf
import pandas as pd
# Data preparation
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt
import yfinance as yf
from yahoofinancials import YahooFinancials
import time
import datetime

In [3]:
#time to pull the data
start_date =datetime.datetime.today().strftime('%Y-%m-%d')
end_date = (datetime.datetime.today() - datetime.timedelta(days=1100)).strftime('%Y-%m-%d')

In [4]:
# calling API 
data=YahooFinancials('tsla').get_historical_price_data(end_date,start_date,"daily")
#into the dataframe
tsla_df = pd.DataFrame(data['TSLA']['prices'])
tsla_df =tsla_df.drop(['high','low','open','volume','date','adjclose'], axis=1)
tsla_df.rename(columns={'formatted_date':'date'},inplace=True)
tsla_df['date']=pd.to_datetime(tsla_df['date'])

In [5]:
# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [1, 2, 3]

In [6]:
tsla_df 

,close,date
0,22.369333,2019-11-07
1,22.476000,2019-11-08
2,23.006001,2019-11-11
3,23.328667,2019-11-12
4,23.073999,2019-11-13
...,...,...
754,207.470001,2022-11-04
755,197.080002,2022-11-07
756,191.300003,2022-11-08
757,177.589996,2022-11-09


In [7]:
# Scale data for ML engine
scaler = MinMaxScaler()
tsla_df ['scaled_close'] = scaler.fit_transform(np.expand_dims(tsla_df['close'].values, axis=1))

In [8]:
def PrepareData(days):
    df = tsla_df.copy()
    df['future'] = df['scaled_close'].shift(-days)
    last_sequence = np.array(df[['scaled_close']].tail(days))
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=N_STEPS)
    
    for entry, target in zip(df[['scaled_close'] + ['date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])
            
    last_sequence = list([s[:len(['scaled_close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # construct the X's and Y's
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

   # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y

In [9]:
PrepareData(3) # 3 days


(          close       date  scaled_close    future
 0     22.369333 2019-11-07      0.001137  0.003610
 1     22.476000 2019-11-08      0.001412  0.002953
 2     23.006001 2019-11-11      0.002778  0.003510
 3     23.328667 2019-11-12      0.003610  0.003994
 4     23.073999 2019-11-13      0.002953  0.003620
 ..          ...        ...           ...       ...
 751  227.820007 2022-11-01      0.530592  0.478149
 752  214.979996 2022-11-02      0.497503  0.451374
 753  215.309998 2022-11-03      0.498353  0.436479
 754  207.470001 2022-11-04      0.478149  0.401147
 755  197.080002 2022-11-07      0.451374  0.434984
 
 [756 rows x 4 columns], array([[0.53239596],
        [0.52987045],
        [0.5305921 ],
        [0.49750283],
        [0.49835327],
        [0.47814927],
        [0.45137382],
        [0.43647853],
        [0.40114728],
        [0.43498385]], dtype=float32), array([[[0.001137335053398629, Timestamp('2019-11-07 00:00:00')],
         [0.0014122191238534754, Timestamp('201

In [10]:
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['scaled_close']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 80

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [14]:
# GET PREDICTIONS
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['scaled_close'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/80
94/94 [==============================] - 8s 18ms/step - loss: 0.0133
Epoch 2/80
94/94 [==============================] - 2s 18ms/step - loss: 0.0040
Epoch 3/80
94/94 [==============================] - 2s 19ms/step - loss: 0.0041
Epoch 4/80
94/94 [==============================] - 1s 12ms/step - loss: 0.0039
Epoch 5/80
94/94 [==============================] - 1s 9ms/step - loss: 0.0028
Epoch 6/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0029
Epoch 7/80
94/94 [==============================] - 1s 9ms/step - loss: 0.0029
Epoch 8/80
94/94 [==============================] - 1s 9ms/step - loss: 0.0027
Epoch 9/80
94/94 [==============================] - 1s 9ms/step - loss: 0.0028
Epoch 10/80
94/94 [==============================] - 1s 9ms/step - loss: 0.0024
Epoch 11/80
94/94 [==============================] - 1s 9ms/step - loss: 0.0024
Epoch 12/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0021
Epoch 13/80
94/94 [========================

1/1 [==============================] - 1s 818ms/step
Epoch 1/80
94/94 [==============================] - 5s 11ms/step - loss: 0.0160
Epoch 2/80
94/94 [==============================] - 1s 11ms/step - loss: 0.0049
Epoch 3/80
94/94 [==============================] - 1s 11ms/step - loss: 0.0049
Epoch 4/80
94/94 [==============================] - 1s 12ms/step - loss: 0.0047
Epoch 5/80
94/94 [==============================] - 1s 11ms/step - loss: 0.0042
Epoch 6/80
94/94 [==============================] - 1s 11ms/step - loss: 0.0044
Epoch 7/80
94/94 [==============================] - 1s 11ms/step - loss: 0.0045
Epoch 8/80
94/94 [==============================] - 1s 11ms/step - loss: 0.0044
Epoch 9/80
94/94 [==============================] - 1s 11ms/step - loss: 0.0036
Epoch 10/80
94/94 [==============================] - 1s 13ms/step - loss: 0.0031
Epoch 11/80
94/94 [==============================] - 2s 16ms/step - loss: 0.0042
Epoch 12/80
94/94 [==============================] - 1s 14ms/step

1/1 [==============================] - 1s 800ms/step


In [15]:
if bool(predictions) == True and len(predictions) > 0:
  predictions_list = [str(d)+'$' for d in predictions]
  predictions_str = ', '.join(predictions_list)
  message = f' prediction for upcoming 3 days ({predictions_str})'
  
  print(message)

 prediction for upcoming 3 days (183.94$, 196.43$, 190.12$)


In [16]:
from google.colab import files
today=start_date =datetime.datetime.today()
day1=(today+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
day2=(today+datetime.timedelta(days=2)).strftime('%Y-%m-%d')
day3=(today+datetime.timedelta(days=3)).strftime('%Y-%m-%d')
days=[day1,day2,day3]
tsla_lstm=pd.DataFrame()
tsla_lstm['future days']=days
tsla_lstm['price']=predictions
tsla_lstm['ticker']='TSLA'

tsla_lstm.to_csv('tsla_lstm.csv') 
files.download('tsla_lstm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>